In [1]:
import json
import importlib
import topicvisexplorer
import pandas as pd
import warnings
import random
import time
import numpy as np
import sys
import re, numpy as np, pandas as pd
import unidecode
import _prepare
import _prepare_single_topic
import gensim_helpers
import spacy
import re, numpy as np, pandas as pd
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
import matplotlib.pyplot as plt

from gensim.models import LdaMulticore
from pprint import pprint
from string import punctuation
from string import digits
from nltk.tokenize import TweetTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
from random import sample


%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)




In [2]:
importlib.reload(topicvisexplorer)
vis = topicvisexplorer.TopicVisExplorer("name")
vis.load_single_corpus_data("models_output/single_corpus_europe_cambridge_analytica_lda_mallet_with_word_embedding_final_dataset_user_study_topics_labeled.pkl")


Data loaded sucessfully


In [3]:
#Import word embedding
single_corpus_data = topicvisexplorer.single_corpus_data

word_embedding_model = single_corpus_data['word_embedding_model']


### topic splitting helpers.py

In [4]:
#/////////////////////////////Parameters////////////////////////////
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['usernameremoved', 'link', 'usernameremove', 'amp', 'linkremoved',' <link removed>','usernameremoved','<usernameremoved>','<linkremoved>','usernameremoved_usernameremoved','linkremoved_linkremoved'])
punctuation+="¡¿<>'`"
punctuation+='"'
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])    
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
tknzr = TweetTokenizer()


name_column_text = 'texto_completo'
name_tokenizacion = 'text_cleaner'
#/////////////////////////////Parameters////////////////////////////

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def sent_to_words(sentence):
    return tknzr.tokenize(sentence)
    
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    doc = nlp(" ".join(texts)) 
    texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
        
def text_cleaner(tweet):

    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()

    tweet = unidecode.unidecode(tweet)
    tweet = sent_to_words(tweet)

    tweet = remove_stopwords(tweet)

    new_tweet  = []
    for elem in tweet:
        if len(elem)>0:
            new_tweet.append(elem[0])
    tweet = lemmatization(new_tweet, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return tweet[0]


def getDocumentVector(text, wordembedding ,  list_terms_relevance):   

    document_vector = np.array([0.0]*300)# 300 dimensions
    
    for word in text:  
        #print('esta es una word', word)
        if word in list_terms_relevance:
            raking_word = float(list_terms_relevance.index(word)+1)
            if word in wordembedding: #if word in wordembedding.wv:
                #print("WORD FOUND", word, raking_word)
                document_vector+=wordembedding[word]/raking_word #aqui hay que ponderar
            else:
                pass
                #print("WARNING, Word not found:", word)    

    return document_vector


def get_initial_document_vector_by_class(list_terms_relevance, topic_id, name_tokenizacion,documents_class_A, documents_class_B, wordembedding):
    relevantDocumentsvector_class_A = 0.0
    list_documents_A = []
    for row in documents_class_A:
        current_text = row[name_tokenizacion]
        current_contribution = row[str(int(topic_id) -1)]
        current_document_vector = getDocumentVector(current_text, wordembedding,  list_terms_relevance)    
        relevantDocumentsvector_class_A+= current_contribution*current_document_vector
        list_documents_A.append((current_contribution, current_text))
    
    relevantDocumentsvector_class_B= 0.0
    list_documents_B = []
    for row in documents_class_B:
        current_text = row[name_tokenizacion]
        current_contribution = row[str(int(topic_id) -1)]
        current_document_vector = getDocumentVector(current_text, wordembedding,  list_terms_relevance)    
        relevantDocumentsvector_class_B+= current_contribution*current_document_vector
        list_documents_B.append((current_contribution, current_text))

    return(relevantDocumentsvector_class_A, relevantDocumentsvector_class_B, list_documents_A, list_documents_B)



def create_two_list_of_documents(list_terms_relevance, list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding):
    vector_A, vector_B, seeds_documents_A, seeds_documents_B = get_initial_document_vector_by_class(list_terms_relevance, topic_id, name_tokenizacion,new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding)

    vector_A = vector_A.reshape(-1, 1)
    vector_B = vector_B.reshape(-1, 1)
    documents_A = []
    documents_B = []
    most_relevant_documents_topic = set()

    list_relevant_documents = pd.DataFrame(list_relevant_documents).sort_values(int(topic_id)-1, ascending=False).reset_index()
    #print('vectores crreadoos', vector_A, vector_B)
    for index, row in list_relevant_documents.head(20000).iterrows():
        if index%100 == 0:
            print(index)
        current_contribution = row[int(topic_id)-1]
        current_text = row[name_tokenizacion]
        current_document_vector = getDocumentVector(current_text, wordembedding, list_terms_relevance).reshape(-1, 1)
        #similarity_vectorA_currentvector =  1 - spatial.distance.cosine(vector_A, current_document_vector)
        #similarity_vectorB_currentvector =  1 - spatial.distance.cosine(vector_B, current_document_vector)
        similarity_vectorA_currentvector = np.arccos(spatial.distance.cosine(vector_A, current_document_vector)-1) / np.pi
        similarity_vectorB_currentvector = np.arccos(spatial.distance.cosine(vector_B, current_document_vector)-1) / np.pi
        
        #I need this information to get the matrix of most relevant documents according to the similarity score
        
        #most_relevant_documents_topic.add((similarity_vectorA_currentvector,similarity_vectorB_currentvector,  current_contribution, row[name_column_text]))
            
        if similarity_vectorA_currentvector>= similarity_vectorB_currentvector:
            #append element to documentsA
            documents_A.append((current_contribution, row[name_tokenizacion]))
            most_relevant_documents_topic.add((similarity_vectorA_currentvector,0,  current_contribution, row[name_column_text]))

        else:
            documents_B.append((current_contribution, row[name_tokenizacion]))
            most_relevant_documents_topic.add((0,similarity_vectorB_currentvector,  current_contribution, row[name_column_text])) #QUIZAS LO CORRECTO Es que en vez de 0, sea 1-similarity_vectorB_currentvector


    print('Documentos en A', len(documents_A))
    print('Documentos en B', len(documents_B))

    #In the set of most relevant documents, we also need to calculate the similarity between the topic vector and the documents as seeds
    for row in new_document_seeds_TopicA:
        current_contribution = row[str(int(topic_id)-1)]
        current_text = row[name_tokenizacion]
        #print('agregando este texto', current_text)
        current_document_vector = getDocumentVector(current_text, wordembedding, list_terms_relevance).reshape(-1, 1)
        #angular distance: https://www.wikiwand.com/en/Cosine_similarity
        #https://math.stackexchange.com/questions/3241174/scale-cosine-similarity-between-vectors-to-range-0-1

        similarity_vectorA_currentvector = np.arccos(spatial.distance.cosine(vector_A, current_document_vector)-1) / np.pi
        #similarity_vectorB_currentvector = np.arccos(spatial.distance.cosine(vector_B, current_document_vector)-1) / np.pi
        #similarity_vectorA_currentvector =  1 - spatial.distance.cosine(vector_A, current_document_vector)
        #similarity_vectorB_currentvector =  1 - spatial.distance.cosine(vector_B, current_document_vector)
        
        most_relevant_documents_topic.add((similarity_vectorA_currentvector, 0, current_contribution, row[name_column_text]))
    
    for row in new_document_seeds_TopicB:
        current_contribution = row[str(int(topic_id)-1)]
        current_text = row[name_tokenizacion]
        #print('agregando este texto', current_text)
        current_document_vector = getDocumentVector(current_text, wordembedding, list_terms_relevance).reshape(-1, 1)
        #similarity_vectorA_currentvector =  1 - spatial.distance.cosine(vector_A, current_document_vector)
        #similarity_vectorB_currentvector =  1 - spatial.distance.cosine(vector_B, current_document_vector)
        #similarity_vectorA_currentvector = np.arccos(spatial.distance.cosine(vector_A, current_document_vector)-1) / np.pi
        similarity_vectorB_currentvector = np.arccos(spatial.distance.cosine(vector_B, current_document_vector)-1) / np.pi
        
        most_relevant_documents_topic.add((0, similarity_vectorB_currentvector, current_contribution, row[name_column_text]))
    
                
    
    return (seeds_documents_A, seeds_documents_B, documents_A, documents_B, most_relevant_documents_topic)

        
def getLdaModelfromSentences(sentences):
    print('lda started')
    #data_words = list(sent_to_words(sentences))
    #data_ready = process_words(sentences)  # processed Text Data!
    # Create Dictionary
    id2word = corpora.Dictionary(sentences)

    # Create Corpus: Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in sentences]
    # Build LDA model
    print('building model')
    lda_model = LdaMulticore(corpus=corpus,
                            id2word=id2word,
                            num_topics=1, 
                            random_state=100,
                            chunksize=10,
                            passes=10,
                            alpha='symmetric',
                            iterations=100,
                            per_word_topics=True)
    print('termine de calcular el modelo')
    return (lda_model, corpus, id2word)


def getCorpusDictionaryfromSentences(sentences):
    print('lda started')
    #data_words = list(sent_to_words(sentences))
    #data_ready = process_words(sentences)  # processed Text Data!
    # Create Dictionary
    id2word = corpora.Dictionary(sentences)

    # Create Corpus: Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in sentences]
    # Build LDA model
    print('sendind corpus and dictionary')
    return (corpus, id2word)


def get_new_subtopics(list_terms_relevance, list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding):
    final_list_A = []
    final_list_B = []

    seeds_documents_A, seeds_documents_B, documents_A, documents_B, most_relevant_documents_topic = create_two_list_of_documents(list_terms_relevance, list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding)
    print('seeds documents a', len(seeds_documents_A))
    print('seeds documents b', len(seeds_documents_B))
    print('seeds documents a', len(documents_A))
    print('seeds documents a', len(documents_B))
    
    for contribution, text in seeds_documents_A:
        final_list_A.extend([text]*int(contribution*100))
    for contribution, text in seeds_documents_B:
        final_list_B.extend([text]*int(contribution*100))

    for contribution, text in documents_A:
        final_list_A.extend([text]*int(contribution*100))
    for contribution, text in documents_B:
        final_list_B.extend([text]*int(contribution*100))
    
    freq_topic_A = len(final_list_A)/(len(final_list_A)+len(final_list_B))
    freq_topic_B = len(final_list_B)/(len(final_list_A)+len(final_list_B))
    
    return(getCorpusDictionaryfromSentences(final_list_A), getCorpusDictionaryfromSentences(final_list_B), most_relevant_documents_topic, freq_topic_A, freq_topic_B)

def update_topic_term_dists(row, total_frequency):
    row['topic_term_dists'] = row['term_frequency']/total_frequency
    return row

    
def extract_data_without_topic_model(corpus, dictionary):


    topic_model = None
    if not gensim.matutils.ismatrix(corpus):
        corpus_csc = gensim.matutils.corpus2csc(corpus, num_terms=len(dictionary))
    else:
        corpus_csc = corpus
        # Need corpus to be a streaming gensim list corpus for len and inference functions below:
        corpus = gensim.matutils.Sparse2Corpus(corpus_csc)

    vocab = list(dictionary.token2id.keys())
    
    beta = 0.01
    fnames_argsort = np.asarray(list(dictionary.token2id.values()), dtype=np.int_)
    term_freqs = corpus_csc.sum(axis=1).A.ravel()[fnames_argsort]
    term_freqs[term_freqs == 0] = beta

    assert term_freqs.shape[0] == len(dictionary),\
        'Term frequencies and dictionary have different shape {} != {}'.format(
        term_freqs.shape[0], len(dictionary))

    topic_term_dists = term_freqs/term_freqs.sum(axis=0) # esta bien esto! 
    return {'topic_term_dists': topic_term_dists, 'vocab': vocab, 'term_frequency': term_freqs}


def change_frequency_on_prepared_data(row, new_subtopic_df, total_sum_frequency_corpus):
    current_term = row['Term']
    current_total = row['Total']
    new_subtopic_df = pd.DataFrame(new_subtopic_df)
    current_total_new_subtopic_df = new_subtopic_df['term_frequency'].sum()
    if current_term in list(new_subtopic_df['vocab']) and current_total>0:
        new_subtopic_df = pd.DataFrame(new_subtopic_df)
        old_freq = row['Freq']
        new_prob = float(new_subtopic_df.loc[new_subtopic_df['vocab'] == current_term]['topic_term_dists'])
        current_frequency = float(new_subtopic_df.loc[new_subtopic_df['vocab'] == current_term]['term_frequency'])

        row['Freq'] = new_prob*row['Total']
        row['logprob'] = np.log(new_prob)
        #row['loglift'] = np.log(new_prob/(current_total/total_sum_frequency_corpus))     
        row['loglift'] = np.log(new_prob/(current_frequency/current_total_new_subtopic_df))                 

    else:
        row['Freq'] = 0    
        row['logprob'] = 0
        row['loglift'] = 0  
    return row
    

    

#Ojo, la frecuencia a actualizar sera del primer parametro q se le pase a la funcion, el q uno llava data model a
#en algun momento ahbra que intercambiar, data model a debe ser data model b. 
def update_current_freq_and_total_freq_on_prepared_data(row, data_model_A_df, data_model_B_df, list_terms_A, list_terms_B, total_sum_frequency_corpus):
    current_term = row['Term']
    if(current_term in list_terms_A):
        term_frequency_A = float(data_model_A_df.loc[data_model_A_df['vocab']==current_term]['term_frequency'])
    else:
        term_frequency_A = 0
    if(current_term in list_terms_B):
        term_frequency_B = float(data_model_B_df.loc[data_model_B_df['vocab']==current_term]['term_frequency'])
    else:
        term_frequency_B = 0
    row['Total'] = term_frequency_A+term_frequency_B
    row['Freq'] = term_frequency_A
    if(current_term in list_terms_A):
        new_prob = float(data_model_A_df.loc[data_model_A_df['vocab'] == current_term]['topic_term_dists'])
        row['logprob'] = np.log(new_prob)
        row['loglift'] = np.log(new_prob/((term_frequency_A+term_frequency_B)/total_sum_frequency_corpus))
    else:
        row['logprob'] = 0
        row['loglift'] = 0 #auqnue la verdad en evz de cero, creo que el valor debiese ser - inf
    return row


    
                                                        
    



In [5]:
pd.set_option('display.max_colwidth', None)


In [6]:
list_relevant_documents = pd.DataFrame(single_corpus_data['relevantDocumentsDict'])

In [7]:
list_relevant_documents[name_tokenizacion] = list_relevant_documents[name_column_text].apply(lambda x: text_cleaner(x))


In [8]:
list_relevant_documents.head()

,0,1,2,3,4,5,6,7,8,9,10,texto_completo,text_cleaner
0,0.067683,0.075224,0.111145,0.083138,0.087522,0.095556,0.066845,0.120091,0.133339,0.066889,0.092569,"Meet our newest product: the transparent CamHatch webcam cover! We've combined everything you loved about the Matte Black version with the transparency of Mark Zuckerberg. Protect your privacy without having to change the appearance of your laptop, tablet or smartphone. <link removed>","[meet, new, product, transparent, camhatch, webcam, cover, combine, love, black, transparency, protect, privacy, change, appearance, laptop, tablet, smartphone, remove]"
1,0.095597,0.098896,0.078370,0.078370,0.078370,0.112853,0.078630,0.078384,0.125669,0.079250,0.095611,"<usernameremoved> This week has been the most #Triggered I've seen Lefties since Trump won (Sadly, we have #Maybot &amp; Amber Dudd on other side) Friends! <link removed>","[week, trigger, see, lefty, trump, sadly, maybot, dudd, side, friend, remove]"
2,0.087413,0.087413,0.087413,0.087643,0.087413,0.087413,0.087413,0.106643,0.106413,0.087413,0.087413,Pulitzer prize for this lady. Investigative journalism at it's finest. <link removed>,"[finest, remove]"
3,0.083831,0.092424,0.136948,0.097352,0.075758,0.109086,0.077989,0.089814,0.075758,0.076336,0.084705,"Theres no more secrecy, confident and privacy and no protection of personal data. Our lives are traded on the open market! <link removed>","[s, confident, privacy, protection, personal, datum, life, trade, open, market, remove]"
4,0.081169,0.148244,0.096213,0.093189,0.081169,0.081169,0.093545,0.081761,0.081204,0.081169,0.081169,AggregateIQ: the obscure Canadian tech firm and the Brexit data riddle <link removed>,"[obscure, canadian, tech, firm, remove]"


In [9]:
PreparedData_dict_with_more_info = single_corpus_data['tinfo_collection']


In [10]:
num_topics = single_corpus_data['lda_model'].num_topics
for i in range(num_topics):
    topic_id=i+1
    list_terms_relevance = PreparedData_dict_with_more_info.loc[PreparedData_dict_with_more_info['Category'] == 'Topic'+str(topic_id)].sort_values(by='relevance', ascending=False)['Term'].tolist()
    list_relevant_documents[str(i)+'_vector'] = list_relevant_documents[name_tokenizacion].apply(lambda x: getDocumentVector(x, word_embedding_model, list_terms_relevance))


In [11]:
list_relevant_documents.to_json('list_relevant_documents_with_vectors_for_topic_splitting.json')

In [12]:
list_relevant_documents.to_pickle('list_relevant_documents_with_vectors_for_topic_splitting.pickle', protocol=4)

In [ ]:
empezo a las 4;50, TERMINO A LAS 6:30!

In [13]:
list_relevant_documents.head()

,0,1,2,3,4,5,6,7,8,9,...,1_vector,2_vector,3_vector,4_vector,5_vector,6_vector,7_vector,8_vector,9_vector,10_vector
0,0.067683,0.075224,0.111145,0.083138,0.087522,0.095556,0.066845,0.120091,0.133339,0.066889,...,"[-0.005832044226394828, 0.004824629630888921, -0.010793960988507934, 0.013014919551096682, 0.013446430498731843, 0.009752103203595652, -0.010154982011954417, -0.020792909589147257, 0.004657446484088723, 0.026906830334041842, 0.006458600246219248, 0.01647228918136534, -0.007870864023061586, 0.004518206507782452, -0.001709750909185459, -0.0007279627780008013, -0.012279298928660864, 0.00107182755004942, 0.010858633869474943, 0.013820536659977734, 0.011039951074053533, -0.007742315181076265, 0.0032889737558434717, 0.01990246420416497, 0.012125395350494728, -0.0009830939028461216, 0.00717800846564387, -0.01226780641309233, 0.01653447000489905, -0.03409489481782657, 0.00027056518530343965, 0.01697082275666162, 0.023338822995810915, -0.007012869494928964, 0.006266129961488787, 0.0009815753284669881, -0.0054750186964156455, 0.021584279062153655, -0.013503107956239546, 0.014206222398797763, 0.00689366975007033, 0.0014186532175273214, 0.005688093851517806, -0.015572400584915158, 0.002687475597667799, -0.026199309231358825, -0.03041700036715156, -0.0035437147271295544, 0.011969313861527553, -0.0030209025705971726, -0.0030674226181872655, -0.01585873337933208, 0.022766084111481177, -0.0018929699263026123, -0.029372355055784283, -0.005523425097180734, -0.006730943033460335, 0.011632306975570827, -0.018044589273358724, -0.0017834583230751377, 0.031042915476973576, 0.015335797264242501, -0.0003667400583253766, 0.011517590729908989, -0.00296702738046406, 0.0272179497484899, 0.009056644198125241, -0.011047980714465666, -0.010318232517050774, 0.005458968934163977, 0.007159763263667074, -0.00017325761064057588, 0.021560793124649535, 0.008283839127784631, 0.0026137264204635358, -0.009203333667187508, 0.0249638945992956, 0.008475949687408502, -0.0008581340023283701, -0.012465410752156458, -0.00035738082851821673, -0.006980544576890679, -0.0028639549893796357, -0.011642599676633836, 0.016928436189118656, 0.00817229218000648, -0.01929266112301775, 0.009595523386622062, -0.04285092310828986, -0.011159990490114069, 0.005951980823056147, 0.012342590735499925, -0.012874009296865552, 0.02894411217846482, -0.0028371057244385156, 0.01156259994456832, -0.007006165677012177, -0.006830357899474393, 0.01261231759247039, -0.0192476599818292, ...]","[0.17334513068948354, 0.4751854061034919, -1.0515528906208829, 0.6605246419176183, -0.8241817232251094, 0.3634195367147868, 1.1369768031418062, -0.4022216703633035, -0.27226905346969943, 1.3428479918347875, -1.1099608783227382, -0.5713002275351755, -0.46185648109394606, -0.5129620459051694, 0.7738794179001616, -1.8590003699431463, -2.060040409215617, 0.06695491959499122, -0.5337720346337846, -1.121600649483753, 0.5275501813157462, -1.095912993811453, 0.1706957257265458, 0.2534024882881454, 1.279347537234571, -0.8141115526568683, -0.7152172798455467, 0.6690871564667304, 1.2226876977865686, -1.124696786536333, 1.2853897268877574, -0.3661356354180043, 0.912875920139868, -0.5869683360961062, -0.18591627318346582, -1.7438747168762347, -0.5077568841768425, -0.3465028613168215, -0.6565510558467622, -1.5070860186744994, -1.3853048407202095, 0.552853267456797, 0.9740750410209102, 1.2033022564123712, 0.35389168694098316, -0.3954737172283975, 0.8668243814928474, -0.5330114154730836, 1.0922150280318874, -0.4282591104556559, -0.045498422900550395, -0.8124023171876615, 1.6854159866916234, -1.216011448644167, -0.14234693556591083, -0.17356662245947518, -0.3262887786844715, 1.0738628061139934, 1.651000260718689, 1.487103464183292, 0.612461379050842, -1.5767503139584278, -0.16098261188903962, -0.38962478720759464, -1.2597194656328838, 0.8776806367662289, -0.045624159546861165, -0.9964637149328155, 0.4301041179799938, 0.9072275350745365, -0.2164295909204128, -0.6560468237535702, 0.7141522949709156, 1.4220121247418547, -1.9036

In [14]:
list_relevant_documents.columns

Index([               0,                1,                2,                3,
                      4,                5,                6,                7,
                      8,                9,               10, 'texto_completo',
         'text_cleaner',       '0_vector',       '1_vector',       '2_vector',
             '3_vector',       '4_vector',       '5_vector',       '6_vector',
             '7_vector',       '8_vector',       '9_vector',      '10_vector'],
      dtype='object')